In [207]:
import pandas as pd
import numpy as np
import requests
import yaml
import time
from selenium import webdriver
import random
import json
from pymongo import MongoClient
from collections import Counter
from selenium.common.exceptions import WebDriverException
import boto3
from collections import defaultdict,Counter

In [137]:
def get_yelp_ids_from_region(locations, n_from_each):
    with open('/Users/ElliottC/.secrets/yelp_keys.txt') as f:
        keys = yaml.load(f)
    headers = {'Authorization': f"Bearer {keys['api_key']}"}

    url = "https://api.yelp.com/v3"
    url_biz = "https://api.yelp.com/v3/businesses/search"
    url_ind_biz = "https://api.yelp.com/v3/businesses/"
    
    client = MongoClient('mongodb://localhost:27017/')
    restaurants = client['restaurants']
    yelp_restaurant_search = restaurants['yelp_restaurant_search']
    for j in range(len(locations)):
        location_name = locations[j]
        print(location_name)
        i = 0
        status = 200
        while (i < int(n_from_each/50)+1) and (status == 200):
            response = requests.get(f"{url_biz}?location={location_name}&offset={i*50}&limit=50&term=restaurant"\
                                    ,params={},headers=headers)
            status = response.status_code
            if status != 200:
                time.sleep(10)
                print(response.json())
            else:
                yelp_restaurant_search.insert_one(response.json())
            print(n_from_each - i*50)
            print(status)
            time.sleep(1)
            i += 1

def flatten_dict(row):
    out = {}
    for key, value in row.items():
        if type(value) != dict:
            out[key] = value
        else:
            sub_key = key
            for k, v in value.items():
                out[sub_key + "|" + k] = v

In [97]:
get_yelp_ids_from_region(seattle_zip_codes,10000)

98104
10000
200
9950
200
9900
200
9850
200
9800
200
9750
200
9700
200
9650
200
9600
200
9550
200
9500
200
9450
200
9400
200
9350
200
9300
200
9250
200
9200
200
9150
200
9100
200
9050
200
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}
9000
400
98154
10000
200
9950
200
9900
200
9850
200
9800
200
9750
200
9700
200
9650
200
9600
200
9550
200
9500
200
9450
200
9400
200
9350
200
9300
200
9250
200
9200
200
9150
200
9100
200
9050
200
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}
9000
400
98101
10000
200
9950
200
9900
200
9850
200
9800
200
9750
200
9700
200
9650
200
9600
200
9550
200
9500
200
9450
200
9400
200
9350
200
9300
200
9250
200
9200
200
9150
200
9100
200
9050
200
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}
9000
400
98121
10000
200
9950
200
9900
200
9850
200
9800
200
9750
200
970

In [126]:
results = pd.DataFrame(list(yelp_restaurant_search.find()))['businesses']

In [129]:
all_results = []
for row in results:
    all_results.extend(row)

In [132]:
results_df = pd.DataFrame(all_results)

In [179]:
def clean_price(row):
    if type(row) == str:
        return len(row)
    else:
        return "not available"

In [180]:
results_df['price'] = results_df['price'].apply(clean_price)

In [201]:
categories = []
for j in range(len(results_df)):
    for i in range(len(results_df['categories'][j])):
        categories.append((results_df['categories'][j][i]['title']))

In [185]:
results_df.sort_values('review_count',ascending=False)

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,pike-place-chowder-seattle,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 47.609411, 'longitude': -122.341191}",(206) 267-2537,1830.230190,6I28wDuMBR5WLMqfKxaoeg,https://s3-media3.fl.yelpcdn.com/bphoto/ijju-w...,False,"{'address1': '1530 Post Aly', 'address2': 'Ste...",Pike Place Chowder,+12062672537,2,4.5,5676,[pickup],https://www.yelp.com/biz/pike-place-chowder-se...
14844,pike-place-chowder-seattle,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 47.609411, 'longitude': -122.341191}",(206) 267-2537,7207.010380,6I28wDuMBR5WLMqfKxaoeg,https://s3-media3.fl.yelpcdn.com/bphoto/ijju-w...,False,"{'address1': '1530 Post Aly', 'address2': 'Ste...",Pike Place Chowder,+12062672537,2,4.5,5676,[pickup],https://www.yelp.com/biz/pike-place-chowder-se...
3793,pike-place-chowder-seattle,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 47.609411, 'longitude': -122.341191}",(206) 267-2537,1180.387708,6I28wDuMBR5WLMqfKxaoeg,https://s3-media3.fl.yelpcdn.com/bphoto/ijju-w...,False,"{'address1': '1530 Post Aly', 'address2': 'Ste...",Pike Place Chowder,+12062672537,2,4.5,5676,[pickup],https://www.yelp.com/biz/pike-place-chowder-se...
3843,pike-place-chowder-seattle,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 47.609411, 'longitude': -122.341191}",(206) 267-2537,1180.387708,6I28wDuMBR5WLMqfKxaoeg,https://s3-media3.fl.yelpcdn.com/bphoto/ijju-w...,False,"{'address1': '1530 Post Aly', 'address2': 'Ste...",Pike Place Chowder,+12062672537,2,4.5,5676,[pickup],https://www.yelp.com/biz/pike-place-chowder-se...
3893,pike-place-chowder-seattle,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 47.609411, 'longitude': -122.341191}",(206) 267-2537,1180.387708,6I28wDuMBR5WLMqfKxaoeg,https://s3-media3.fl.yelpcdn.com/bphoto/ijju-w...,False,"{'address1': '1530 Post Aly', 'address2': 'Ste...",Pike Place Chowder,+12062672537,2,4.5,5676,[pickup],https://www.yelp.com/biz/pike-place-chowder-se...
2150,pike-place-chowder-seattle,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 47.609411, 'longitude': -122.341191}",(206) 267-2537,1830.230190,6I28wDuMBR5WLMqfKxaoeg,https://s3-media3.fl.yelpcdn.com/bphoto/ijju-w...,False,"{'address1': '1530 Post Aly', 'address2': 'Ste...",Pike Place Chowder,+12062672537,2,4.5,5676,[pickup],https://www.yelp.com/biz/pike-place-chowder-se...
3943,pike-place-chowder-seattle,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 47.609411, 'longitude': -122.341191}",(206) 267-2537,1180.387708,6I28wDuMBR5WLMqfKxaoeg,https://s3-media3.fl.yelpcdn.com/bphoto/ijju-w...,False,"{'address1': '1530 Post Aly', 'address2': 'Ste...",Pike Place Chowder,+12062672537,2,4.5,5676,[pickup],https://www.yelp.com/biz/pike-place-chowder-se...
4343,pike-place-chowder-seattle,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 47.609411, 'longitude': -122.341191}",(206) 267-2537,1180.387708,6I28wDuMBR5WLMqfKxaoeg,https://s3-media3.fl.yelpcdn.com/bphoto/ijju-w...,False,"{'address1': '1530 Post Aly', 'address2': 'Ste...",Pike Place Chowder,+12062672537,2,4.5,5676,[pickup],https://www.yelp.com/biz/pike-place-chowder-se...
4293,pike-place-chowder-seattle,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 47.609411, 'longitude': -122.341191}",(206) 267-2537,1180.387708,6I28wDuMBR5WLMqfKxaoeg,https://s3-media3.fl.yelpcdn.com/bphoto/ijju-w...,False,"{'address1': '1530 Post Aly', 'address2': 'Ste...",Pike Place Chowder,+12062672537,2,4.5,5676,[pickup],https://www.yelp.com/biz/pike-place-chowder-se...
8515,pike-place-chowder-seattle,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 47.609411, 'longitude': -122.341191}",(206) 267-2537,2268.275443,6I28wDuMBR5WLMqfKxaoeg,https://s3-media3.fl.yelpcdn.com/bphoto/ijju-w...,False,"{'address1': '1530 Post Aly', 'address2': 'Ste...",Pike 

In [148]:
results_df['categories'][0]

[{'alias': 'seafood', 'title': 'Seafood'},
 {'alias': 'tradamerican', 'title': 'American (Traditional)'},
 {'alias': 'soup', 'title': 'Soup'}]

In [106]:
url_ind_biz = "https://api.yelp.com/v3/businesses/"

In [ ]:
response = requests.get(url_ind_biz+'/',params={},headers=headers)

In [104]:
new_yelp_df['businesses'][0][0]

{'alias': 'pike-place-chowder-seattle',
 'categories': [{'alias': 'seafood', 'title': 'Seafood'},
  {'alias': 'tradamerican', 'title': 'American (Traditional)'},
  {'alias': 'soup', 'title': 'Soup'}],
 'coordinates': {'latitude': 47.609411, 'longitude': -122.341191},
 'display_phone': '(206) 267-2537',
 'distance': 1830.2301898696153,
 'id': '6I28wDuMBR5WLMqfKxaoeg',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/ijju-wYoRAxWjHPTCxyQGQ/o.jpg',
 'is_closed': False,
 'location': {'address1': '1530 Post Aly',
  'address2': 'Ste 11',
  'address3': '',
  'city': 'Seattle',
  'country': 'US',
  'display_address': ['1530 Post Aly', 'Ste 11', 'Seattle, WA 98101'],
  'state': 'WA',
  'zip_code': '98101'},
 'name': 'Pike Place Chowder',
 'phone': '+12062672537',
 'price': '$$',
 'rating': 4.5,
 'review_count': 5676,
 'transactions': ['pickup'],
 'url': 'https://www.yelp.com/biz/pike-place-chowder-seattle?adjust_creative=jC7u_3tIMTa2Gq5o4EZ7oQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_bu

In [72]:
seattle_zip_codes = ['98104','98154','98101','98121','98122','98109',
                     '98102','98122','98109','98119','98199','98107','98103','98105',]

In [73]:
response.json()

{'businesses': [{'alias': 'maono-fried-chicken-seattle',
   'categories': [{'alias': 'sandwiches', 'title': 'Sandwiches'},
    {'alias': 'chickenshop', 'title': 'Chicken Shop'}],
   'coordinates': {'latitude': 47.66251, 'longitude': -122.29969},
   'display_phone': '',
   'distance': 556.1867909448827,
   'id': '0gWFjjSxi9hdAhA7UqSf5w',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/7g18pscCl01d3c_NKrorPw/o.jpg',
   'is_closed': False,
   'location': {'address1': '4626 26th Ave NE',
    'address2': '',
    'address3': None,
    'city': 'Seattle',
    'country': 'US',
    'display_address': ['4626 26th Ave NE', 'Seattle, WA 98105'],
    'state': 'WA',
    'zip_code': '98105'},
   'name': "Ma'ono Fried Chicken",
   'phone': '',
   'price': '$$',
   'rating': 4.0,
   'review_count': 101,
   'transactions': [],
   'url': 'https://www.yelp.com/biz/maono-fried-chicken-seattle?adjust_creative=jC7u_3tIMTa2Gq5o4EZ7oQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_sourc

In [66]:
get_yelp_ids_from_region('seattle',10000)

10000
200
9950
200
9900
200
9850
200
9800
200
9750
200
9700
200
9650
200
9600
200
9550
200
9500
200
9450
200
9400
200
9350
200
9300
200
9250
200
9200
200
9150
200
9100
200
9050
200
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}
9000
400
{'error': {'code': 'TOO_LARGE_VALIDATION_ERROR', 'description': 'Number too big: 1050'}}
8950
400
{'error': {'code': 'TOO_LARGE_VALIDATION_ERROR', 'description': 'Number too big: 1100'}}
8900
400


KeyboardInterrupt: 

In [50]:
int(220/50)

4

In [6]:
with open('/Users/ElliottC/.secrets/yelp_keys.txt') as f:
    keys = yaml.load(f)
headers = {'Authorization': f"Bearer {keys['api_key']}"}

url = "https://api.yelp.com/v3"
url_biz = "https://api.yelp.com/v3/businesses/search"
url_ind_biz = "https://api.yelp.com/v3/businesses/"

In [33]:
for i in range(n_to_scrape):
    response = requests.get(f"{url_biz}?location=seattle&offset={i*50}&limit=50",params={},headers=headers)
    response

In [79]:
response = requests.get(f"{url_biz}?location=seattle&offset={1*50}&limit=50",params={},headers=headers)

In [80]:
len(response.json().keys())

3

In [51]:
client = MongoClient('mongodb://localhost:27017/')
restaurants = client['restaurants']
current_yelp_restaurants = restaurants['current_yelp_restaurants']

In [ ]:
current_yelp_restaurants.insert_one()

In [7]:
def scrape_individual(list_of_ids, start, n_to_scrape, short_sleep=1, long_sleep=10, params={}):
#     client = MongoClient('mongodb://localhost:27017/')
#     db = client.lb_tst1
#     businesses = db.lb_tst1
    output = []
    for i in range(start, start + n_to_scrape):
        response = requests.get(url_ind_biz + list_of_ids[i], params=params, headers=headers)
        if response.status_code != 200:
            print(response.status_code)
            time.sleep(long_sleep)
        time.sleep(short_sleep)
        try:
            output.append(response.json())
        except:
            print('already exists in database')
        if i % 50 == 0:
            print(i)
#     end_idx = i + 1
    return end_idx, output